In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.sql("select * from retail_databricks_catalog.silver_schema.customer_silver")
df.display()

In [0]:
df = df.dropDuplicates(subset=['customer_id'])
df.display()

### Surrogate Key


In [0]:
df = df.withColumn("DimCustomerKey" , monotonically_increasing_id()+lit(1))
df.display()